In [31]:
from search import SearchQA
from typing import Union
from fastapi import FastAPI
app = FastAPI()

@app.get("/items/age")
async def read_age(age: int):
    return {"age": age}



search_qa = SearchQA()


query = """
Instant Ramen
"""


input = {
    'age' : "40",
    'pregnancy_week': "20",
    'input': query
}

response = search_qa.get_result(input['age'], input['pregnancy_week'], input['input'])

2024-08-10 15:16:44,091 - 139722656323392 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-10 15:16:44][DEBUG]: == Working Agent: Food and Nutrient for Pregnant Women Specialist
 [2024-08-10 15:16:44][INFO]: == Starting Task: 
        You are tasked with providing detailed nutritional information and recommendations for pregnant women based on their specific query. Analyze the user's input, consider any mentioned trimester or specific concerns, and provide a comprehensive response.
        
        <user_input>
        - User (Pregnant woman) age: 40
        - 20 week pregnancy
        - Food search query: 
Instant Ramen

        </user_input> 
        
        Use the search tool to gather the most up-to-date and accurate information. Ensure your response is tailored to the needs of pregnant women and follows the expected output format.
        
        Assess the food safety level using these guidelines. You must consider the pregnancy trimester to assess the safety level into 1 to 5.
        <safety_level_guideline>

        Level 1: Optimal Foods
        High i

In [32]:
response.raw

'1. Food Item Analysis:\n\n    - Name: Instant Ramen\n    - Safety level: 4 (Foods to Limit)\n    - Nutritional Value: High in calories, carbohydrates, and sodium; low in essential vitamins and fiber\n    - Recommended Intake: Moderate consumption (1-2 servings per week)\n    - Trimester-specific Recommendations: 2nd trimester - Limit intake due to high sodium content; 3rd trimester - Avoid excessive consumption to minimize risk of high blood pressure\n\n2. Key nutrients and calories calculation\n\n    | Nutrient      | Amount per 100g | % Daily Value* |\n    |---------------|-----------------|----------------|\n    | Calories      | 470 kcal        | 23.5%          |\n    | Carbohydrates | 60g             | 20%            |\n    | Protein       | 10g             | 20%            |\n    | Fat           | 17g             | 26%            |\n    | Sodium        | 990-1400mg      | 41-58%          |\n    | Sugar         | 2g              | 4%             |\n\n3. Benefits for Pregnancy:\n\

In [33]:
import re

def extract_safety_level(response):
    match = re.search(r'Safety level:\s*(\d+)', response)
    
    if match:
        # 찾은 숫자를 정수로 변환하여 반환합니다.
        return int(match.group(1))
    else:
        # 안전 수준을 찾지 못한 경우 None을 반환합니다.
        return None

extract_safety_level(response=response.raw) #! connect to API

4

In [34]:
import re

def parse_by_section(text):
    # 섹션을 분리하는 정규표현식을 더 유연하게 수정
    section_pattern = r'(\d+\.?\s*[^:\n]+):\s*([\s\S]+?)(?=\n(?:\d+\.?\s|$))'
    sections = re.findall(section_pattern, text, re.MULTILINE)
    
    result = {}
    for title, content in sections:
        # 키 생성: 번호 제거, 소문자 변환, 공백을 언더스코어로 변경
        key = re.sub(r'^\d+\.?\s*', '', title).strip().lower().replace(' ', '_')
        # 내용에서 앞뒤 공백 제거
        value = content.strip()
        result[key] = value
    
    return result

# 사용 예시:
parsed_data = parse_by_section(response.raw) #! connect to API


In [35]:
import re
import pandas as pd
import io

def extract_key_nutrients_section(text):
    pattern = r'2\.\s*Key nutrients and calories calculation\s*([\s\S]+?)(?=\n\d+\.\s|\Z)'
    match = re.search(pattern, text)
    
    if match:
        content = match.group(1).strip()
        return content
    else:
        return None

def markdown_table_to_dataframe(markdown_table):
    # Markdown 표를 문자열 IO로 변환
    table_io = io.StringIO(markdown_table)
    
    # pandas로 읽기
    df = pd.read_csv(table_io, sep='|', skipinitialspace=True)
    
    # 첫 번째와 마지막 열(비어있는 열) 제거
    df = df.iloc[:, 1:-1]
    
    # 열 이름 정리
    df.columns = df.columns.str.strip()
    
    # 첫 번째 행(구분선) 제거
    df = df.iloc[1:]
    
    # 인덱스 재설정
    df = df.reset_index(drop=True)
    
    return df


markdown_table_to_dataframe(extract_key_nutrients_section(response.raw)) #! connect to API

,Nutrient,Amount per 100g,% Daily Value*
0,Calories,470 kcal,23.5%
1,Carbohydrates,60g,20%
2,Protein,10g,20%
3,Fat,17g,26%
4,Sodium,990-1400mg,41-58%
5,Sugar,2g,4%


In [36]:
parsed_data

{'food_item_analysis': '- Name: Instant Ramen\n    - Safety level: 4 (Foods to Limit)\n    - Nutritional Value: High in calories, carbohydrates, and sodium; low in essential vitamins and fiber\n    - Recommended Intake: Moderate consumption (1-2 servings per week)\n    - Trimester-specific Recommendations: 2nd trimester - Limit intake due to high sodium content; 3rd trimester - Avoid excessive consumption to minimize risk of high blood pressure',
 'benefits_for_pregnancy': '- Instant Ramen can be a convenient and occasional source of carbohydrates and calories during pregnancy.',
 'potential_risks_or_contraindications': '- Excessive consumption of Instant Ramen may lead to high sodium intake, potentially increasing the risk of high blood pressure and other complications during pregnancy.',
 'preparation_tips': '- To make Instant Ramen safer for consumption during pregnancy, add fresh vegetables, lean proteins, and other nutrient-rich foods to enhance its nutritional value.',
 'alternat